Carlos,
      In the dataset, there are conversations between workers and ChatGPT or between workers and lawyers on Zoom. I would like to ask you for help with the following:

1) **Separate what text corresponds to the worker and which to the IA (or the lawyer).**

2) Extract measures of quality from the conversation.

**a) How complete was the response, did it cover many aspects of the question?**
b) How easy to understand was the response
c) Relevant:  That is: did it actually respond to the question asked?
d) How empathic the response was, that is:
was the name of the worker mentioned? how many times?
Was it friendly
Did the IA or lawyer made the worker feel understood?
e) How long did the conversation last (not possible now since we only have duration for the lawyer zoom data)


Thanks!!
Enirque

In [2]:
import pandas as pd
import pandas as pd
import numpy as np
import json
import requests
import os
import pickle
from google.colab import auth, drive
import time
auth.authenticate_user()
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
df = pd.read_stata("gdrive/MyDrive/Conciliación laboral/conversations_ai.dta")

In [4]:
df2 = pd.read_stata("gdrive/MyDrive/Conciliación laboral/conversations_human.dta")

In [5]:
df

,id,fecha_creacion,fecha_inicio_sesion,fecha_fin_sesion,motivo_finalizacion,duracion_sesion,numero_preguntas,preguntas_mostradas,preguntas_seleccionadas,conversacion,session_id
0,480,10/21/2025 12:26,10/21/2025 12:26,10/21/2025 12:26,usuario,00:00:00,1,"[4,10,7,2]",[4],### Mensaje de la Persona\n\n¿Cuánto tiempo te...,564.0
1,479,10/21/2025 12:02,10/21/2025 12:02,10/21/2025 12:02,usuario,00:00:00,1,"[10,6,11,1]",[],### Mensaje de la Persona\n\nQue pasa si no ll...,67.0
2,478,10/21/2025 12:00,10/21/2025 12:00,10/21/2025 12:00,,00:00:00,1,"[11,6,2,7]",[6],### Mensaje de la Persona\n\n¿Qué prestaciones...,486.0
3,477,10/21/2025 11:32,10/21/2025 11:32,10/21/2025 11:35,,00:02:44,3,"[1,2,7,4]",[2],### Mensaje de la Persona\n\n¿Cuál debería de ...,419.0
4,476,10/21/2025 11:32,10/21/2025 11:32,10/21/2025 11:35,usuario,00:03:25,3,"[4,5,2,11,3,8]",[4],### Mensaje de la Persona\n\n¿Cuánto tiempo te...,445.0
...,...,...,...,...,...,...,...,...,...,...,...
620,1027,11/11/2025 7:58,11/11/2025 7:58,11/11/2025 8:02,usuario,00:03:54,4,"[9,8,11,3]","[9,8,11,3]",### Mensaje de la Persona\n\n¿Me recomiendas l...,18.0
621,1026,11/11/2025 7:54,11/11/2025 7:54,11/11/2025 7:58,usuario,00:03:55,5,"[6,7,3,11]","[6,3,11]",### Mensaje de la Persona\n\n¿Qué prestaciones...,588.0
622,1025,11/11/2025 7:54,11/11/2025 7:54,11/11/2025 7:58,usuario,00:03:25,4,"[6,2,11,3]","[3,2,6,11]",### Mensaje de la Persona\n\n¿Cuál es la difer...,382.0
623,1024,11/11/2025 7:37,11/11/2025 7:37,11/11/2025 7:42,usuario,00:04:34,7,"[7,4,11,6]",[6],### Mensaje de la Persona\n\n¿Qué prestaciones...,103.0


In [ ]:
df2[df2["session_id"] == 1]

,fecha_creacion_liga,estado,fecha_atencion_asesor,usuario_toma_sesion,usuario_zoom_utilizado,fecha_sesion_iniciada,fecha_sesion_finalizada,motivo_finalizacion,duracion_sesion,preguntas_mostradas,preguntas_seleccionadas,resumen,transcripcion,session_id
577,11/10/2025 10:02,finalizado,11/10/2025 10:02,Christian Mendoza,asesor3@ilj.mx,11/10/2025 10:02,11/10/2025 10:19,webhook,00:17:14,"[3,7,10,11]",[3],## Resumen\n\n### Asesoría sobre Conflicto Lab...,WEBVTT\n\n00:00:33.000 --> 00:00:35.000\nChris...,1.0


Juntar ambas bases

Dividir en columnas ASESOR y USUARIO

Meter ai a las preguntas

In [6]:
df2

,fecha_creacion_liga,estado,fecha_atencion_asesor,usuario_toma_sesion,usuario_zoom_utilizado,fecha_sesion_iniciada,fecha_sesion_finalizada,motivo_finalizacion,duracion_sesion,preguntas_mostradas,preguntas_seleccionadas,resumen,transcripcion,session_id
0,10/21/2025 12:37,finalizado,10/21/2025 12:38,Galia Hernández,asesor1@ilj.mx,10/21/2025 12:38,10/21/2025 12:59,webhook,00:21:34,[],[],,,453.0
1,10/21/2025 12:35,finalizado,10/21/2025 12:35,Antonio Carbonel,asesor4@ilj.mx,10/21/2025 12:35,10/21/2025 12:48,webhook,00:12:46,[],[],## Resumen\n\n### Derechos por Despido Injusti...,WEBVTT\n\n00:00:23.000 --> 00:00:25.000\nAseso...,384.0
2,10/21/2025 12:34,finalizado,10/21/2025 12:34,Galia Hernández,asesor1@ilj.mx,10/21/2025 12:35,10/21/2025 12:38,webhook,00:03:21,[],[],,,287.0
3,10/21/2025 12:12,finalizado,10/21/2025 12:13,Christian Mendoza,asesor2@ilj.mx,10/21/2025 12:13,10/21/2025 12:27,webhook,00:13:44,[],[],,,136.0
4,10/21/2025 12:12,finalizado,10/21/2025 12:12,Galia Hernández,asesor1@ilj.mx,10/21/2025 12:12,10/21/2025 12:32,webhook,00:19:35,[],[],,,372.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
636,11/11/2025 8:08,finalizado,11/11/2025 8:08,Andrea Ferreiro,asesor4@ilj.mx,11/11/2025 8:08,11/11/2025 8:22,manual_con_asesoria,00:13:44,"[3,6,8,11]","[6,3,8]",## Resumen\n\n### Consultoría Proceso del Cent...,WEBVTT\n\n00:00:36.000 --> 00:00:38.000\nAseso...,497.0
637,11/11/2025 7:46,finalizado,11/11/2025 7:46,Andrea Ferreiro,asesor4@ilj.mx,11/11/2025 7:46,11/11/2025 8:03,manual_con_asesoria,00:16:46,"[2,3,10,11]",[3],## Resumen\n\n### Conciliación Laboral y Proce...,WEBVTT\n\n00:00:31.000 --> 00:00:33.000\nAseso...,380.0
638,11/11/2025 7:44,finalizado,11/11/2025 7:44,Alma Rosales,asesor2@ilj.mx,11/11/2025 7:44,11/11/2025 7:58,manual_con_asesoria,00:13:22,"[1,2,4,5]","[1,2]",## Resumen\n\n### Orientación y Selección de P...,WEBVTT\n\n00:00:36.000 --> 00:00:40.000\nAseso...,49.0
639,11/11/2025 7:44,finalizado,11/11/2025 7:44,Cecilia Cruz,asesor3@ilj.mx,11/11/2025 7:44,11/11/2025 8:02,manual_con_asesoria,00:18:00,"[4,5,8,11]",[],## Resumen\n\n### Consultoría Laboral y Despid...,WEBVTT\n\n00:01:00.000 --> 00:01:02.000\nLeona...,97.0


LIMPIEZA

In [ ]:
df.conversacion.sample(1).iloc[0]

'### Mensaje de la Persona\n\n¿Cuánto tiempo tengo para exigir mis derechos?\n\n### Respuesta del Asistente\n\n🎯 **Tiempo para Exigir Derechos por Despido**\n\nCuando una persona trabajadora es despedida, el tiempo es un factor crucial para proteger sus derechos. Según la **Ley Federal del Trabajo** (LFT) en el artículo 518, tienes un plazo de **dos meses** desde el día siguiente en que te separaron de tu trabajo para exigir tus derechos.\n\n## 🗓️ Detalles Importantes\n\n- **Inicio del Plazo**: El conteo comienza al día siguiente del despido. Por ejemplo, si te despidieron el 1 de enero, tienes hasta el 2 de marzo.\n- **Meses Calendario**: Es importante tener en cuenta que estos dos meses se consideran como meses calendario, no se cuentan como 60 días.\n- **Pausa del Plazo**: Si presentas una solicitud de conciliación, este plazo se "pausa" mientras dura el procedimiento de conciliación prejudicial.\n\n## 📋 Ejemplo de Pausa en el Plazo\n\n| Fecha | Evento | Días transcurridos del plazo

In [ ]:
df.conversacion.isna().sum()

np.int64(0)

In [ ]:
df.columns

Index(['id', 'fecha_creacion', 'fecha_inicio_sesion', 'fecha_fin_sesion',
       'motivo_finalizacion', 'duracion_sesion', 'numero_preguntas',
       'preguntas_mostradas', 'preguntas_seleccionadas', 'conversacion',
       'session_id'],
      dtype='object')

In [ ]:
import os
import re
import json
import pandas as pd
from pathlib import Path


DEST_ROOT = Path("gdrive/MyDrive/Conciliación laboral/Conversaciones ai")

OUTPUT_FORMAT = "csv"

COL_CONV = "conversacion"

COL_ID = "id"

COL_SESSION_ID = "session_id"

In [ ]:
df2.sample().iloc[0].transcripcion

"WEBVTT\n\n00:00:29.000 --> 00:00:40.000\nAsesor Laboral Cuatro: Hola, buenos dias. Me ayudaria aprendiendo, por favor, el m microfono, opiviendo le a mi companero que esta hay cerca sindi ayuda, por favor, aprendo el microfono.\n\n00:00:42.000 --> 00:00:44.000\nRosa: Listen. Albert Croquin, yeah.\n\n00:00:43.000 --> 00:00:49.000\nAsesor Laboral Cuatro: Perfecto. Si. Si, es acto, ya la escucho muy bien. Buenos dia, rosa como esta.\n\n00:00:46.000 --> 00:00:48.000\nRosa: Okay.\n\n00:00:48.000 --> 00:00:52.000\nRosa: Buenos dias. Vienna que mire. Tondo lata.\n\n00:00:51.000 --> 00:01:06.000\nAsesor Laboral Cuatro: Queue. No, no es nada de la. Me presento, mi nombre es alm, pues yo voy a tratar de ayudarle en esta pequena sesion que vamos a tener 15 minutos, pues justa como a resolverle algunas dudas.\n\n00:01:07.000 --> 00:01:21.000\nAsesor Laboral Cuatro: Respecto al procedimiento de conciliación respecto a qué es lo que viene a hacer el día de hoy aquí al centro de conciliación vale si

In [ ]:
df2.columns

Index(['fecha_creacion_liga', 'estado', 'fecha_atencion_asesor',
       'usuario_toma_sesion', 'usuario_zoom_utilizado',
       'fecha_sesion_iniciada', 'fecha_sesion_finalizada',
       'motivo_finalizacion', 'duracion_sesion', 'preguntas_mostradas',
       'preguntas_seleccionadas', 'resumen', 'transcripcion', 'session_id'],
      dtype='object')

In [ ]:
df2.sample(1).transcripcion.iloc[0]

"WEBVTT\n\n00:00:23.000 --> 00:00:25.000\nAsesor Laboral Tres: Hola. Buen día me escucha.\n\n00:00:28.000 --> 00:00:35.000\nAsesor Laboral Tres: Buen día. No sé si me escuche, pero yo a usted no le escucho. No sé si podría apoyarme con el micrófono o decirle al compa perfecto. Ahí está.\n\n00:00:35.000 --> 00:00:36.000\nclaudia: Sí, Sí. Sí. Escuchándome.\n\n00:00:36.000 --> 00:00:38.000\nAsesor Laboral Tres: Hola, Hola, ¿qué tal buen día? ¿cómo está.\n\n00:00:39.000 --> 00:00:42.000\nclaudia: Buen día. Bien, Gracias, y usted.\n\n00:00:41.000 --> 00:00:47.000\nAsesor Laboral Tres: Perfecto. Me presento muy bien. Muchísimas gracias. Le present, le presento mi nombre. Es Cristian Mendoza.\n\n00:00:42.000 --> 00:00:44.000\nclaudia: Y a.\n\n00:00:47.000 --> 00:00:52.000\nAsesor Laboral Tres: Y el día de hoy, a el día de hoy le voy a dar una asesoría aproximadamente como de 12\xa0min.\n\n00:00:52.000 --> 00:00:55.000\nAsesor Laboral Tres: Para comenzar, me podría corroborar su nombre. Por fa

In [ ]:
df2

,fecha_creacion_liga,estado,fecha_atencion_asesor,usuario_toma_sesion,usuario_zoom_utilizado,fecha_sesion_iniciada,fecha_sesion_finalizada,motivo_finalizacion,duracion_sesion,preguntas_mostradas,preguntas_seleccionadas,resumen,transcripcion,session_id
0,10/21/2025 12:37,finalizado,10/21/2025 12:38,Galia Hernández,asesor1@ilj.mx,10/21/2025 12:38,10/21/2025 12:59,webhook,00:21:34,[],[],,,453.0
1,10/21/2025 12:35,finalizado,10/21/2025 12:35,Antonio Carbonel,asesor4@ilj.mx,10/21/2025 12:35,10/21/2025 12:48,webhook,00:12:46,[],[],## Resumen\n\n### Derechos por Despido Injusti...,WEBVTT\n\n00:00:23.000 --> 00:00:25.000\nAseso...,384.0
2,10/21/2025 12:34,finalizado,10/21/2025 12:34,Galia Hernández,asesor1@ilj.mx,10/21/2025 12:35,10/21/2025 12:38,webhook,00:03:21,[],[],,,287.0
3,10/21/2025 12:12,finalizado,10/21/2025 12:13,Christian Mendoza,asesor2@ilj.mx,10/21/2025 12:13,10/21/2025 12:27,webhook,00:13:44,[],[],,,136.0
4,10/21/2025 12:12,finalizado,10/21/2025 12:12,Galia Hernández,asesor1@ilj.mx,10/21/2025 12:12,10/21/2025 12:32,webhook,00:19:35,[],[],,,372.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
636,11/11/2025 8:08,finalizado,11/11/2025 8:08,Andrea Ferreiro,asesor4@ilj.mx,11/11/2025 8:08,11/11/2025 8:22,manual_con_asesoria,00:13:44,"[3,6,8,11]","[6,3,8]",## Resumen\n\n### Consultoría Proceso del Cent...,WEBVTT\n\n00:00:36.000 --> 00:00:38.000\nAseso...,497.0
637,11/11/2025 7:46,finalizado,11/11/2025 7:46,Andrea Ferreiro,asesor4@ilj.mx,11/11/2025 7:46,11/11/2025 8:03,manual_con_asesoria,00:16:46,"[2,3,10,11]",[3],## Resumen\n\n### Conciliación Laboral y Proce...,WEBVTT\n\n00:00:31.000 --> 00:00:33.000\nAseso...,380.0
638,11/11/2025 7:44,finalizado,11/11/2025 7:44,Alma Rosales,asesor2@ilj.mx,11/11/2025 7:44,11/11/2025 7:58,manual_con_asesoria,00:13:22,"[1,2,4,5]","[1,2]",## Resumen\n\n### Orientación y Selección de P...,WEBVTT\n\n00:00:36.000 --> 00:00:40.000\nAseso...,49.0
639,11/11/2025 7:44,finalizado,11/11/2025 7:44,Cecilia Cruz,asesor3@ilj.mx,11/11/2025 7:44,11/11/2025 8:02,manual_con_asesoria,00:18:00,"[4,5,8,11]",[],## Resumen\n\n### Consultoría Laboral y Despid...,WEBVTT\n\n00:01:00.000 --> 00:01:02.000\nLeona...,97.0


# Funciones

In [ ]:
import re
from typing import Union, Dict
import pandas as pd

def _normalize_session_id(sid):
    """
    Normaliza session_id para que sea un string usable:
    - Si es float y representa un entero (167.0) -> "167"
    - Si es NaN/None -> "row_<index>"
    - Si es otra cosa -> str(sid)
    """
    if sid is None:
        return None
    try:
        # pandas NA check
        if pd.isna(sid):
            return None
    except Exception:
        pass
    # si es float que es entero, convierto a int
    if isinstance(sid, float):
        if sid.is_integer():
            return str(int(sid))
        else:
            return str(sid)
    return str(sid)

def parse_webvtt_transcription(
    df: pd.DataFrame,
    row_identifier: Union[None, int, str] = None,
    trans_col: str = "transcripcion",
    session_col: str = "session_id",
    id_col: str = "session_id"
) -> Union[pd.DataFrame, Dict[str, pd.DataFrame]]:
    """
    Parsea transcripciones WEBVTT -> DataFrame(s) con columnas:
      participante, iteracion, texto, tiempo

    Normaliza session_id (ej. 167.0 -> "167") para keys/nombres de archivo.
    Si la transcripción total tiene < 10 palabras retorna DataFrame con NA en las columnas.
    """
    timestamp_re = re.compile(
        r'(?P<start>\d{2}:\d{2}:\d{2}\.\d{3})\s*-->\s*(?P<end>\d{2}:\d{2}:\d{2}\.\d{3})'
    )

    def _parse_single_transcript(text: str):
        if not isinstance(text, str) or text.strip() == "":
            return []
        lines = text.splitlines()
        results = []
        i = 0
        n = len(lines)
        while i < n:
            line = lines[i].strip()
            m = timestamp_re.search(line)
            if m:
                start = m.group("start")
                end = m.group("end")
                tiempo = f"{start} --> {end}"
                i += 1
                cue_lines = []
                while i < n:
                    nxt = lines[i].rstrip("\n")
                    # si la siguiente línea contiene timestamp, es nuevo cue
                    if timestamp_re.search(nxt):
                        break
                    # si línea vacía: la saltamos (separador)
                    if nxt.strip() == "":
                        i += 1
                        continue
                    cue_lines.append(nxt.strip())
                    i += 1
                cue_text = " ".join(cue_lines).strip()
                participante = None
                texto = cue_text
                if cue_text:
                    mname = re.match(r'^\s*([^:]{1,200}):\s*(.*)$', cue_text)
                    if mname:
                        participante = mname.group(1).strip()
                        texto = mname.group(2).strip()
                    else:
                        participante = "unknown"
                else:
                    participante = "unknown"
                    texto = ""
                results.append({
                    "participante": participante,
                    "texto": texto,
                    "tiempo": tiempo
                })
            else:
                i += 1
        return results

    def _build_df_from_row(row, row_index=None):
        trans_text = row.get(trans_col, "")
        parsed = _parse_single_transcript(trans_text)

        # contar palabras totales
        total_words = sum(len(str(p.get("texto","")).split()) for p in parsed)

        if total_words < 10:
            return pd.DataFrame([{
                "participante": pd.NA,
                "iteracion": pd.NA,
                "texto": pd.NA,
                "tiempo": pd.NA
            }], columns=["participante", "iteracion", "texto", "tiempo"])

        rows = []
        for idx, item in enumerate(parsed, start=1):
            rows.append({
                "participante": item.get("participante"),
                "iteracion": idx,
                "texto": item.get("texto"),
                "tiempo": item.get("tiempo")
            })
        return pd.DataFrame(rows, columns=["participante", "iteracion", "texto", "tiempo"])

    # procesar todo o una fila
    if row_identifier is None:
        out = {}
        for index, row in df.iterrows():
            sid_raw = row.get(id_col)
            sid_norm = _normalize_session_id(sid_raw)
            if sid_norm is None:
                sid_norm = f"row_{index}"
            out[sid_norm] = _build_df_from_row(row, row_index=index)
        return out
    else:
        # buscar por índice
        if isinstance(row_identifier, int) and row_identifier in df.index:
            row = df.loc[row_identifier]
            return _build_df_from_row(row, row_index=row_identifier)
        # buscar por coincidencia en id_col (considerando que el id en df puede ser float)
        # comparamos normalizados para evitar mismatches tipo 167 vs 167.0
        target_norm = None
        # intentar normalizar el row_identifier
        try:
            target_norm = _normalize_session_id(row_identifier)
        except Exception:
            target_norm = str(row_identifier)
        # buscar recorriendo filas y comparando normalizados
        for index, row in df.iterrows():
            sid_raw = row.get(id_col)
            sid_norm = _normalize_session_id(sid_raw)
            if sid_norm is None:
                continue
            if sid_norm == target_norm:
                return _build_df_from_row(row, row_index=index)
        # fallback: si row_identifier es posición numérica válida
        if isinstance(row_identifier, int) and 0 <= row_identifier < len(df):
            row = df.iloc[row_identifier]
            return _build_df_from_row(row, row_index=row_identifier)

        raise KeyError(f"No se encontró la fila con row_identifier={row_identifier!r}")

In [ ]:
import re
import pandas as pd
from typing import Union, Dict

def conversation_to_dataframe(
    df: pd.DataFrame,
    row_identifier: Union[None, int, str] = None,
    id_col: str = "id",
    conv_col: str = "conversacion",
    session_col: str = "session_id"
) -> Union[pd.DataFrame, Dict[str, pd.DataFrame]]:
    """
    Extrae iteraciones de conversación (persona <-> ai) desde la columna `conv_col`
    y devuelve un DataFrame con columnas: iteracion, PParticipante, id_sesion, texto.

    Parámetros:
    - df: DataFrame que contiene (al menos) las columnas id_col, conv_col y session_col.
    - row_identifier:
        * si es None -> se procesa todo el DataFrame y se devuelve dict {id: df_conversacion}
        * si es int y coincide con un índice de fila -> procesa esa fila por índice
        * si es str/int y coincide con un valor en la columna id_col -> procesa la fila con ese id
    - id_col: nombre de la columna con el id (usa para nombrar archivos o para buscar fila)
    - conv_col: nombre de la columna que contiene la conversación
    - session_col: columna que contiene el session_id (se añade en cada fila del resultado)

    Retorna:
    - pd.DataFrame (si se pidió una sola conversación) o dict de DataFrames (si row_identifier is None)
    """
    def _parse_conv_text(text: str):
        """Devuelve lista de (participant, texto) en orden de aparición."""
        if not isinstance(text, str) or text.strip() == "":
            return []
        # Captura bloques encabezados por "### <titulo>" hasta el siguiente "###" o EOF
        pattern = re.compile(r'###\s*(.+?)\s*\n(.*?)(?=(?:\n###|$))', flags=re.IGNORECASE | re.DOTALL)
        matches = pattern.findall(text)
        parsed = []
        if matches:
            for head, body in matches:
                head_l = head.lower()
                if "persona" in head_l or "usuario" in head_l or "mensaje" in head_l and "persona" in head_l:
                    participant = "persona"
                elif "asistente" in head_l or "assistant" in head_l or "ai" in head_l:
                    participant = "ai"
                else:
                    # heurística: si contiene 'mensaje' y not 'asistente' -> persona
                    if "mensaje" in head_l and "asistente" not in head_l:
                        participant = "persona"
                    else:
                        participant = "unknown"
                parsed.append((participant, body.strip()))
        else:
            # fallback: no hay encabezados "###" -> intentar separar por dobles saltos de línea
            blocks = [b.strip() for b in re.split(r'\n\s*\n', text) if b.strip()]
            # asignar alternancia empezando por persona (mejor que dejar todo unknown)
            part = "persona"
            for b in blocks:
                parsed.append((part, b))
                part = "ai" if part == "persona" else "persona"
        return parsed

    def _build_df_from_row(row):
        conv_text = row.get(conv_col, "")
        session_id = row.get(session_col, "")
        parsed = _parse_conv_text(conv_text)
        # crear filas
        rows = []
        for i, (participant, body) in enumerate(parsed, start=1):
            rows.append({
                "iteracion": i,
                "PParticipante": participant,
                "id_sesion": session_id,
                "texto": body
            })
        if not rows:
            # fila vacía si no hay contenido
            rows = [{"iteracion": 1, "PParticipante": "unknown", "id_sesion": session_id, "texto": ""}]
        return pd.DataFrame(rows, columns=["iteracion", "PParticipante", "id_sesion", "texto"])

    # manejar row_identifier
    if row_identifier is None:
        # procesar todas las filas -> devolver dict {id_value: df_conversacion}
        results = {}
        for _, row in df.iterrows():
            id_val = row.get(id_col)
            if pd.isna(id_val) or id_val == "":
                # generar id por índice si falta
                id_val = f"row_{_}"
            results[str(id_val)] = _build_df_from_row(row)
        return results
    else:
        # intentar tratar row_identifier como índice numérico primero
        if isinstance(row_identifier, int) and row_identifier in df.index:
            row = df.loc[row_identifier]
            return _build_df_from_row(row)
        # si coincide con un valor en la columna id_col
        mask = df[id_col].astype(str) == str(row_identifier)
        if mask.any():
            row = df.loc[mask].iloc[0]
            return _build_df_from_row(row)
        # fallback: si row_identifier es int pero no índice, intentar por posición
        if isinstance(row_identifier, int) and 0 <= row_identifier < len(df):
            row = df.iloc[row_identifier]
            return _build_df_from_row(row)
        raise KeyError(f"No se encontró la fila con row_identifier={row_identifier!r}")

# Pipeline

In [ ]:
all_dict = conversation_to_dataframe(df)

In [ ]:
len(all_dict)

625

In [ ]:
df_conv_123 = conversation_to_dataframe(prueba, row_identifier="494")

In [ ]:
df_conv_123

,iteracion,PParticipante,id_sesion,texto
0,1,persona,205.0,¿Cuál es la diferencia entre un despido justif...
1,2,ai,205.0,🎯 **Diferencia entre Despido Justificado e Inj...
2,3,persona,205.0,Me lo explicas
3,4,ai,205.0,"Claro, Marco Antonio, aquí tienes una explicac..."


In [ ]:
df[df["id"] == 1023].conversacion.iloc[0]

'### Mensaje de la Persona\n\n¿Cuál es la diferencia entre un despido justificado y uno injustificado?\n\n### Respuesta del Asistente\n\n🎯 **Descifrando el Despido: Justificado vs Injustificado**\n\nCuando una persona trabajadora enfrenta un despido, es crucial entender si este se considera justificado o injustificado, ya que eso afecta lo que puede reclamar. Vamos a verlo de manera sencilla:\n\n## 📋 Puntos Clave\n\n1. **Despido Justificado**\n   - Un despido es considerado justificado cuando ocurre por alguna de las razones que se mencionan en el artículo 47 de la Ley Federal del Trabajo. Algunos ejemplos son:\n     - *Robar o dañar deliberadamente propiedad de la empresa.*\n     - *No acatar órdenes sin un motivo válido.*\n     - *Faltar al trabajo injustificadamente seguido.*\n   - En estos casos, la empresa debe informar al trabajador sobre las razones del despido.\n\n2. **Despido Injustificado**\n   - Si el empleador no puede demostrar que se cumplieron estas razones o no avisa al

In [ ]:
for id_value in df["id"]:

    guardado = conversation_to_dataframe(df, row_identifier=id_value)

    # Guardar en JSON
    guardado.to_json(
        f"gdrive/MyDrive/Conciliación laboral/Conversaciones ai/{id_value}.json",
        orient="records",
        force_ascii=False,
        indent=2
    )

In [ ]:
sample = df2

In [ ]:
df2

,fecha_creacion_liga,estado,fecha_atencion_asesor,usuario_toma_sesion,usuario_zoom_utilizado,fecha_sesion_iniciada,fecha_sesion_finalizada,motivo_finalizacion,duracion_sesion,preguntas_mostradas,preguntas_seleccionadas,resumen,transcripcion,session_id
0,10/21/2025 12:37,finalizado,10/21/2025 12:38,Galia Hernández,asesor1@ilj.mx,10/21/2025 12:38,10/21/2025 12:59,webhook,00:21:34,[],[],,,453.0
1,10/21/2025 12:35,finalizado,10/21/2025 12:35,Antonio Carbonel,asesor4@ilj.mx,10/21/2025 12:35,10/21/2025 12:48,webhook,00:12:46,[],[],## Resumen\n\n### Derechos por Despido Injusti...,WEBVTT\n\n00:00:23.000 --> 00:00:25.000\nAseso...,384.0
2,10/21/2025 12:34,finalizado,10/21/2025 12:34,Galia Hernández,asesor1@ilj.mx,10/21/2025 12:35,10/21/2025 12:38,webhook,00:03:21,[],[],,,287.0
3,10/21/2025 12:12,finalizado,10/21/2025 12:13,Christian Mendoza,asesor2@ilj.mx,10/21/2025 12:13,10/21/2025 12:27,webhook,00:13:44,[],[],,,136.0
4,10/21/2025 12:12,finalizado,10/21/2025 12:12,Galia Hernández,asesor1@ilj.mx,10/21/2025 12:12,10/21/2025 12:32,webhook,00:19:35,[],[],,,372.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
636,11/11/2025 8:08,finalizado,11/11/2025 8:08,Andrea Ferreiro,asesor4@ilj.mx,11/11/2025 8:08,11/11/2025 8:22,manual_con_asesoria,00:13:44,"[3,6,8,11]","[6,3,8]",## Resumen\n\n### Consultoría Proceso del Cent...,WEBVTT\n\n00:00:36.000 --> 00:00:38.000\nAseso...,497.0
637,11/11/2025 7:46,finalizado,11/11/2025 7:46,Andrea Ferreiro,asesor4@ilj.mx,11/11/2025 7:46,11/11/2025 8:03,manual_con_asesoria,00:16:46,"[2,3,10,11]",[3],## Resumen\n\n### Conciliación Laboral y Proce...,WEBVTT\n\n00:00:31.000 --> 00:00:33.000\nAseso...,380.0
638,11/11/2025 7:44,finalizado,11/11/2025 7:44,Alma Rosales,asesor2@ilj.mx,11/11/2025 7:44,11/11/2025 7:58,manual_con_asesoria,00:13:22,"[1,2,4,5]","[1,2]",## Resumen\n\n### Orientación y Selección de P...,WEBVTT\n\n00:00:36.000 --> 00:00:40.000\nAseso...,49.0
639,11/11/2025 7:44,finalizado,11/11/2025 7:44,Cecilia Cruz,asesor3@ilj.mx,11/11/2025 7:44,11/11/2025 8:02,manual_con_asesoria,00:18:00,"[4,5,8,11]",[],## Resumen\n\n### Consultoría Laboral y Despid...,WEBVTT\n\n00:01:00.000 --> 00:01:02.000\nLeona...,97.0


In [ ]:
all_parsed = parse_webvtt_transcription(sample)

In [ ]:
big_df = pd.concat(
    {sid: df2 for sid, df2 in all_parsed.items()},
    names=["session_id"]
).reset_index(level=0).reset_index(drop=True)

In [ ]:
df2[df2["session_id"] == 387.0]

,fecha_creacion_liga,estado,fecha_atencion_asesor,usuario_toma_sesion,usuario_zoom_utilizado,fecha_sesion_iniciada,fecha_sesion_finalizada,motivo_finalizacion,duracion_sesion,preguntas_mostradas,preguntas_seleccionadas,resumen,transcripcion,session_id
18,10/21/2025 10:27,finalizado,10/21/2025 10:27,Tamara Perdomo,asesor1@ilj.mx,10/21/2025 10:27,10/21/2025 10:41,webhook,00:13:31,[],[],,,387.0
54,10/21/2025 10:27,finalizado,10/21/2025 10:27,Tamara Perdomo,asesor1@ilj.mx,10/21/2025 10:27,10/21/2025 10:41,webhook,00:13:31,[],[],,,387.0
90,10/21/2025 10:27,finalizado,10/21/2025 10:27,Tamara Perdomo,asesor1@ilj.mx,10/21/2025 10:27,10/21/2025 10:41,webhook,00:13:31,[],[],,,387.0
126,10/21/2025 10:27,finalizado,10/21/2025 10:27,Tamara Perdomo,asesor1@ilj.mx,10/21/2025 10:27,10/21/2025 10:41,webhook,00:13:31,[],[],,,387.0


In [ ]:
big_df["session_id"].nunique()

533

In [ ]:
big_df

,session_id,participante,iteracion,texto,tiempo
0,453,NaN,<NA>,NaN,NaN
1,384,Asesor Laboral Cuatro,1,"That's fun to me, right?",00:00:23.000 --> 00:00:25.000
2,384,Asesor Laboral Cuatro,2,"No, decidos en escuchar deja vez puedo desacti...",00:00:43.000 --> 00:00:47.000
3,384,MARIO,3,"Pisto, croquet.",00:00:54.000 --> 00:00:56.000
4,384,Asesor Laboral Cuatro,4,"I hate this couture, Mario.",00:00:55.000 --> 00:00:57.000
...,...,...,...,...,...
93477,7,Damarus,196,No glass settings.,00:14:34.000 --> 00:14:36.000
93478,7,Asesor Laboral Uno,197,Aplausos.,00:14:35.000 --> 00:14:37.000
93479,7,Damarus,198,Wonderful.,00:14:36.000 --> 00:14:38.000
93480,7,Asesor Laboral Uno,199,Pues sería todo de mi parte. Damaris. Muchísim...,00:14:39.000 --> 00:14:43.000


In [ ]:
for session_id, df_conv in big_df.groupby("session_id"):

    id_value = str(int(session_id))  # por si viene como 167.0 → "167"

    output_path = f"gdrive/MyDrive/Conciliación laboral/Conversaciones abogados/{id_value}.json"

    df_conv.to_json(
        output_path,
        orient="records",
        force_ascii=False,
        indent=2
    )